<a href="https://colab.research.google.com/github/Shaguftha29/Dst/blob/main/FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# If your file is named differently (like credit_card_fraud_dataset.csv), just put that name below ↓
df = pd.read_csv('credit_card_fraud_dataset.csv')
print(df.shape)
df.head()

(100000, 7)


,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
0,1,15:35.5,4189.27,688,refund,San Antonio,0
1,2,20:35.5,2659.71,109,refund,Dallas,0
2,3,08:35.5,784.00,394,purchase,New York,0
3,4,50:35.5,3514.40,944,purchase,Philadelphia,0
4,5,51:35.5,369.07,475,purchase,Phoenix,0


In [ ]:
# install imbalance-learn if not present
!pip install imbalanced-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [ ]:
import pandas as pd

df = pd.read_csv('credit_card_fraud_dataset.csv')
df.head()

,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
0,1,15:35.5,4189.27,688,refund,San Antonio,0
1,2,20:35.5,2659.71,109,refund,Dallas,0
2,3,08:35.5,784.00,394,purchase,New York,0
3,4,50:35.5,3514.40,944,purchase,Philadelphia,0
4,5,51:35.5,369.07,475,purchase,Phoenix,0


In [ ]:
df.columns
df.dtypes

,0
TransactionID,int64
TransactionDate,object
Amount,float64
MerchantID,int64
TransactionType,object
Location,object
IsFraud,int64


In [ ]:
print(df.shape)
print(df.columns)

(100000, 7)
Index(['TransactionID', 'TransactionDate', 'Amount', 'MerchantID',
       'TransactionType', 'Location', 'IsFraud'],
      dtype='object')


In [ ]:
df = pd.read_csv('credit_card_fraud_dataset.csv', header=None)
df.columns = ['Transaction','TransactionDate','amount','MerchantID','TransactionType','Location','IsFraud']  # exactly 7

In [ ]:
df.head()


,Transaction,TransactionDate,amount,MerchantID,TransactionType,Location,IsFraud
0,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
1,1,15:35.5,4189.27,688,refund,San Antonio,0
2,2,20:35.5,2659.71,109,refund,Dallas,0
3,3,08:35.5,784,394,purchase,New York,0
4,4,50:35.5,3514.4,944,purchase,Philadelphia,0


In [ ]:
import pandas as pd

df = pd.read_csv('credit_card_fraud_dataset.csv', header=None)
df.columns = ['Transaction','TransactionDate','amount','MerchantID',
              'TransactionType','Location','IsFraud']

df.head()
print(df['IsFraud'].value_counts())

from sklearn.preprocessing import StandardScaler
# Force 'amount' column to numeric & drop bad rows
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')  # convert anything not numeric -> NaN
df = df.dropna(subset=['amount'])                            # drop rows where amount is NaN
df['amount'] = df['amount'].astype(float)                    # ensure float type

# Now safely scale
scaler = StandardScaler()
df['amount'] = scaler.fit_transform(df[['amount']])

# One-hot encode all categorical columns
df_model = pd.get_dummies(df, columns=['Transaction','TransactionDate','MerchantID',
                                       'TransactionType','Location'], drop_first=True)

# Split X and y
X = df_model.drop('IsFraud', axis=1)
y = df_model['IsFraud']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=42,
                                                    stratify=y)



IsFraud
0          99000
1           1000
IsFraud        1
Name: count, dtype: int64


/tmp/ipython-input-2749220332.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amount'] = df['amount'].astype(float)                    # ensure float type
/tmp/ipython-input-2749220332.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amount'] = scaler.fit_transform(df[['amount']])


In [ ]:


#------------ (a) Random Undersampling ------------
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)
print("Undersampling distribution:\n", y_train_rus.value_counts())

In [ ]:
# ---------------- Logistic Regression ----------------
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report   # <-- this import was missing

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_rus, y_train_rus)
pred_lr = lr.predict(X_test)

print("\n=== Logistic (Undersampled) ===")
print(confusion_matrix(y_test, pred_lr))
print(classification_report(y_test, pred_lr))


=== Logistic (Undersampled) ===
[[4831 4551]
 [  48   47]]
              precision    recall  f1-score   support

           0       0.99      0.51      0.68      9382
           1       0.01      0.49      0.02        95

    accuracy                           0.51      9477
   macro avg       0.50      0.50      0.35      9477
weighted avg       0.98      0.51      0.67      9477



In [ ]:
# ---------------- Random Forest ----------------
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_rus, y_train_rus)
pred_rf = rf.predict(X_test)
print("\n=== Random Forest (Undersampled) ===")
print(confusion_matrix(y_test, pred_rf))
print(classification_report(y_test, pred_rf))


=== Random Forest (Undersampled) ===
[[5160 4222]
 [  49   46]]
              precision    recall  f1-score   support

           0       0.99      0.55      0.71      9382
           1       0.01      0.48      0.02        95

    accuracy                           0.55      9477
   macro avg       0.50      0.52      0.36      9477
weighted avg       0.98      0.55      0.70      9477



In [ ]:
# ---------------- SVM ----------------
from sklearn.svm import SVC
svm = SVC(kernel='rbf')
svm.fit(X_train_rus, y_train_rus)
pred_svm = svm.predict(X_test)
print("\n=== SVM (Undersampled) ===")
print(confusion_matrix(y_test, pred_svm))
print(classification_report(y_test, pred_svm))



=== SVM (Undersampled) ===
[[5114 4268]
 [  47   48]]
              precision    recall  f1-score   support

           0       0.99      0.55      0.70      9382
           1       0.01      0.51      0.02        95

    accuracy                           0.54      9477
   macro avg       0.50      0.53      0.36      9477
weighted avg       0.98      0.54      0.70      9477

